# 向量存储（即向量数据库）
* 在非常快速的可搜索数据库中存储嵌入。

## Setup

## 创建你的 .env 文件
* 在 GitHub 仓库中，我们包含了一个名为 .env.example 的文件
* 将该文件重命名为 .env 文件，这里是你添加机密 API 密钥的地方。记得包括：
* OPENAI_API_KEY=你的_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=你的_langchain_api_key
* LANGCHAIN_PROJECT=你的项目名称

我们将我们的 LangSmith 项目称为 **004-vector-stores**。

## Connect with the .env file located in the same directory of this notebook

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

## 提醒：RAG 过程的步骤。
* 当您加载文档时，最终会得到字符串。有时这些字符串会太大，无法适应上下文窗口。在这种情况下，我们将使用 RAG 技术：
    * 将文档拆分成小块。
    * 将文本块转换为数值块（嵌入）。
    * **将嵌入加载到向量数据库（即向量存储）中**。
    * 加载问题并检索最相关的嵌入以进行回答。
    * 将嵌入发送给 LLM 以正确格式化响应。

## 向量数据库（即向量存储）：存储和搜索嵌入
* 查看文档页面 [在这里](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)。
* 查看向量存储的列表 [在这里](https://python.langchain.com/v0.1/docs/integrations/vectorstores/)。

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
loaded_document = TextLoader('./data/state_of_the_union.txt').load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

chunks_of_text = text_splitter.split_documents(loaded_document)

vector_db = Chroma.from_documents(chunks_of_text, OpenAIEmbeddings())

In [8]:
question = "What did the president say about the John Lewis Voting Rights Act?"

response = vector_db.similarity_search(question)

print(response[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


在之前的代码中，`.similarity_search` 方法用于从已处理的文档中查找与给定查询相似的最相关文本块。以下是按步骤的工作原理：

1. **文档处理和嵌入：**
   - 使用 LangChain Community 包中的 `TextLoader` 加载文档 `state_of_the_union.txt`。
   - 然后使用 `CharacterTextSplitter` 将文档拆分为较小的文本块，每个块长 1000 个字符，且块之间没有重叠。
   - 每个文本块随后通过 `OpenAIEmbeddings` 转换为嵌入。嵌入是表示文本语义内容的高维向量。
   - 这些嵌入存储在 `Chroma` 的实例中，它作为一个优化了高效相似性搜索的向量数据库。

2. **使用 `.similarity_search`：**
   - 当您调用 `vector_db.similarity_search(question)` 时，该方法使用与文本块相同的方法将查询（`"总统关于约翰·刘易斯投票权法案说了什么？"`）转换为嵌入。
   - 然后在向量数据库（`vector_db`）中搜索，与查询的嵌入最相似的文本块的嵌入。该相似性通常使用余弦相似度等指标进行测量。
   - 搜索结果按相关性排序，最相关的文本块（与查询在语义上最接近的文本块）首先返回。

3. **输出：**
   - `.similarity_search` 的结果保存在 `response` 中，包含相关文本块及其相似性分数。
   - 脚本打印出最相关文本块的内容（第一个结果），理想情况下应包含总统关于约翰·刘易斯投票权法案的言论信息。

此方法在问答或文档检索等应用中尤其有用，您需要根据查询快速找到大型文本中最相关的部分。